In [1]:
from google.cloud import bigquery
import func
import numpy as np
import pandas as pd
import importlib
from numba import jit

Client creating using default project: tensile-quasar-352901
Target dataset: predict_future_sales


# 1st submission
使用特徴量: block, item_id, shop_id, item_category_id

In [2]:
# トレーニングデータ取得->月ごとに整形。
sql_train = """
SELECT 
    tbl1.date_block_num as block,
    tbl1.item_id,
    tbl1.shop_id,
    tbl2.item_category_id,
    CAST(SUM(tbl1.item_cnt_day) AS INTEGER) as item_cnt_month
FROM `predict_future_sales.sales_train` tbl1
LEFT JOIN `predict_future_sales.items` tbl2
ON tbl1.item_id = tbl2.item_id
GROUP BY block, item_id, shop_id, item_category_id
"""
train = func.get_data(sql_train)
train

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,block,item_id,shop_id,item_category_id,item_cnt_month
0,0,21567,23,38,2
1,0,21771,23,19,2
2,0,21747,23,38,1
3,0,21766,23,40,1
4,0,20299,23,37,1
...,...,...,...,...,...
1609119,33,2293,59,28,2
1609120,33,1523,59,21,1
1609121,33,5913,59,30,1
1609122,33,5658,59,3,1


In [3]:
# テストデータ取得＆特徴量追加
sql_test = """
SELECT 
    34 as block,
    tbl1.shop_id,
    tbl1.item_id,
    tbl2.item_category_id
FROM `predict_future_sales.test` tbl1
LEFT JOIN `predict_future_sales.items` tbl2
ON tbl1.item_id = tbl2.item_id
"""
test = func.get_data(sql_test)
test

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,block,shop_id,item_id,item_category_id
0,34,5,4869,22
1,34,5,5823,35
2,34,5,5643,2
3,34,5,5820,35
4,34,5,5821,35
...,...,...,...,...
214195,34,45,856,78
214196,34,45,849,78
214197,34,45,846,78
214198,34,45,843,78


In [4]:
# クロスバリデーション
accuracies =  func.cross_validate(train)
for i,accuracy in enumerate(accuracies):
    print(f"training block: [0...{32-i}], validation block: {[33-i]}, RMSE: {accuracy}")

training block: [0...32], validation block: [33], RMSE: 2.4358368721492547
training block: [0...31], validation block: [32], RMSE: 2.2570799574788385
training block: [0...30], validation block: [31], RMSE: 1.7997529850136316
training block: [0...29], validation block: [30], RMSE: 1.7934309748421742
training block: [0...28], validation block: [29], RMSE: 2.1609445010486064
training block: [0...27], validation block: [28], RMSE: 2.334465104682721
training block: [0...26], validation block: [27], RMSE: 2.336839390732882
training block: [0...25], validation block: [26], RMSE: 2.0360214170769306
training block: [0...24], validation block: [25], RMSE: 2.1250743245471906
training block: [0...23], validation block: [24], RMSE: 2.345887071028407


In [5]:
# テストをpredictしてsubmissionファイルを作成
func.make_submission(train, test, "submission1.csv")

# 2nd submission

使用特徴量: block, item_id, shop_id, item_category_id, from_first_release

In [6]:
# item_cnt_monthをテストデータに合わせて、[0,20]にクリップ
train["item_cnt_month"] = np.clip(train.item_cnt_month, 0, 20)

In [7]:
# first_release_block: 商品ごとに初めて商品が売れたblock(月)
first_release_block = np.full(22170, 34)
for item_id, block in train.groupby('item_id').head(1)[["item_id", "block"]].values:
    first_release_block[item_id] = block

In [8]:
# 特徴量追加(トレーニング) from_first_release: 初めて商品が売れてから経過した月数
train["from_first_release"] = train.block - first_release_block[train.item_id]

In [9]:
# 各店・商品についてfirst_releaseから現在まで、購買データのない月は、その月その店で売れなかったとして
# item_cnt_monthが0の行をトレーニングデータに追加（ただし、その店でその商品が売れたことがある場合に限る）
@jit(nopython=True)
def additional_row(train_np):
    additional_row = []
    for sid in np.unique(train_np[:, 2]):
        arr = train_np[train_np[:,2]==sid]
        for iid in np.unique(arr[:,1]):
            arr2 = arr[arr[:,1]==iid]
            icid = arr2[0,3]
            start_block = arr2[0,0]
            start_ffr = arr2[0,5]
            end_ffr = start_ffr + 34 - start_block
            for ffr in range(start_ffr, end_ffr):
                if not ffr in arr2[:,5]:
                    additional_row.append([start_block+ffr-start_ffr, iid, sid, icid, 0, ffr])
    return additional_row

# trainをnumpy化
train_np = np.array(train)
# a_row: 追加するデータのリスト、a_df: データフレーム化
a_row = additional_row(train_np)
a_df = pd.DataFrame(np.stack(a_row), columns=["block","item_id","shop_id","item_category_id","item_cnt_month","from_first_release"])
# 追加後の新しいトレーニングデータ
new_train = pd.concat([a_df, train]).reset_index(drop=True)
new_train

,block,item_id,shop_id,item_category_id,item_cnt_month,from_first_release
0,2,30,0,40,0,1
1,3,30,0,40,0,2
2,4,30,0,40,0,3
3,5,30,0,40,0,4
4,6,30,0,40,0,5
...,...,...,...,...,...,...
9532032,33,2293,59,28,2,11
9532033,33,1523,59,21,1,33
9532034,33,5913,59,30,1,5
9532035,33,5658,59,3,1,9


In [10]:
# クロスバリデーション
accuracies =  func.cross_validate(new_train)
for i,accuracy in enumerate(accuracies):
    print(f"training block: [0...{32-i}], validation block: {[33-i]}, RMSE: {accuracy}")

training block: [0...32], validation block: [33], RMSE: 0.7585145714619589
training block: [0...31], validation block: [32], RMSE: 0.7464648533856657
training block: [0...30], validation block: [31], RMSE: 0.6972310470850177
training block: [0...29], validation block: [30], RMSE: 0.6780579944797386
training block: [0...28], validation block: [29], RMSE: 0.7178896090652669
training block: [0...27], validation block: [28], RMSE: 0.7584711803302076
training block: [0...26], validation block: [27], RMSE: 0.810750676085434
training block: [0...25], validation block: [26], RMSE: 0.8315405299211176
training block: [0...24], validation block: [25], RMSE: 0.8405866127509151
training block: [0...23], validation block: [24], RMSE: 1.0211076187821906


In [11]:
# 特徴量追加(テスト) from_first_release: 初めて商品が売れてから経過した月数
test["from_first_release"] = test.block - first_release_block[test.item_id]

In [12]:
# テストをpredictしてsubmissionファイルを作成
func.make_submission(new_train, test, "submission2.csv")

# 3rd submission

使用特徴量: item_id, shop_id, item_category_id, from_first_release

In [13]:
# (3rd submission)特徴量 block を 削除
new_train = new_train.drop('block', axis=1)
test = test.drop('block', axis=1)

In [14]:
# テストをpredictしてsubmissionファイルを作成
func.make_submission(new_train, test, "submission3.csv")

# 4th submission

使用特徴量: item_id, shop_id, item_category_id, from_first_release

In [15]:
# (4th submission) テストに出てこないitem を 削除
new_train = new_train[new_train.item_id.isin(test.item_id.unique())]

In [16]:
# テストをpredictしてsubmissionファイルを作成
func.make_submission(new_train, test, "submission4.csv")